In [7]:
import import_ipynb

import sys

from Dynamic_resistancelib import *

import scipy as sc
import numpy as np

print("Python version: ", sys.version)
print("numpy version: ", np.__version__)

Python version:  3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
numpy version:  1.17.2


In [8]:
####-------------------------------------------------------------------------------------------------------####
#
# TKI only until Prog
#
####-------------------------------------------------------------------------------------------------------####

def Dynamic_TKIonlyUntilProg_MetsDRRLRR_CoMorbid_local(i):

    # select index of model parameter (i.e. define patient)
    N0_total=GN0_total[i]
    N0_persister_frac=GN0_persister_frac[i]
    N0_resistance=GN0_resistance[i]
    r_sen=Gr_sen[i]
    r_per=Gr_per[i]
    r_res=Gr_res[i]
    beta=Gbeta[i]
    
    h0=np.array([N0_total*(1-N0_persister_frac)-N0_resistance,N0_resistance,N0_total*N0_persister_frac])
    p_mutation = G_mu_set
    
    h0_local = h0
    h0_dist = (Gmet_scl)*h0
    h0 = (1+Gmet_scl)*h0
        
    # Lists to store results
    # total,local,distant
    cellSensTreated = np.zeros([18010,3]); #(30*12*5+1)*10
    cellPerTreated = np.zeros([18010,3]);
    cellResTreated = np.zeros([18010,3]);
    cellTotalTreated = np.zeros([18010,3]);    
    erlotinibPlasma = np.zeros(18010); 

    cellSensTreated[0,0] = h0[0];
    cellPerTreated[0,0] =  h0[2];
    cellResTreated[0,0] =  h0[1];
    cellTotalTreated[0,0] = np.sum(h0); 
    
    cellSensTreated[0,1] = h0_local[0];
    cellPerTreated[0,1] =  h0_local[2];
    cellResTreated[0,1] =  h0_local[1];
    cellTotalTreated[0,1] = np.sum(h0_local); 
    
    cellSensTreated[0,2] = h0_dist[0];
    cellPerTreated[0,2] =  h0_dist[2];
    cellResTreated[0,2] =  h0_dist[1];
    cellTotalTreated[0,2] = np.sum(h0_dist); 
    
    ####### TKI Until Progression  #######
    
    #t definition of TIME
    tInterval = 0.1
    tTKI = np.arange(tInterval, 30*12*5+1+tInterval/10, tInterval)
    daysTKI = 30*12*5+1  # Number of days to simulate (8 wks)
    
    # loops in days until progression but calculates for every 0.1 day using odeint
    Cured=False;t_cure = 18000;
    for step in range(0,np.size(tTKI)-1,10):
        intTime = np.arange(0,1,tInterval)
        # Get dose for this step
        
        # get initial values for first 0.1 time point with the TKI daily bolus
        doseErlotinibDaily = 150 
        if step>1:
            erlotinibStart =  sc.integrate.odeint(Pharmacokinetics, h0[3], [0,1], args=([1.0182 + 0.0114*doseErlotinibDaily], ))
            erlotinibPlasma[step] = erlotinibStart[1]
            cellsTreat_local = sc.integrate.odeint(EGFRkineticsGompertz_opt2, h0_local[0:3], [0, 1], args=(Vlimit, r_sen/10, r_res/10, r_per/10, p_mutation, beta,erlotinibStart[1]))
            cellsTreat_dist = sc.integrate.odeint(EGFRkineticsGompertz_opt2, h0_dist[0:3], [0, 1], args=(Vlimit, r_sen/10, r_res/10, r_per/10, p_mutation, beta,erlotinibStart[1]))

            cellSensTreated[step,:]=np.array([float(cellsTreat_local[1,0])+float(cellsTreat_dist[1,0]), float(cellsTreat_local[1,0]), float(cellsTreat_dist[1,0])])
            cellResTreated[step,:]= np.array([float(cellsTreat_local[1,1])+float(cellsTreat_dist[1,1]), float(cellsTreat_local[1,1]), float(cellsTreat_dist[1,1])])
            cellPerTreated[step,:]= np.array([float(cellsTreat_local[1,2])+float(cellsTreat_dist[1,2]), float(cellsTreat_local[1,2]), float(cellsTreat_dist[1,2])])

        # Update initial condition for next step
        h0_local =  np.array([cellSensTreated[step,1], cellResTreated[step,1], cellPerTreated[step,1],erlotinibPlasma[step]])
        h0_dist =   np.array([cellSensTreated[step,2], cellResTreated[step,2], cellPerTreated[step,2],erlotinibPlasma[step]])
        h0 = h0_local+h0_dist; h0[3] = h0[3]/2; # because doubled when local and distant addded
        
        # Solve the ODEs (Combined Gompertz Growth/TKI Kill and & Pharmacokinetics over 1 day)
        doseinterval=0
        cellsTreat_local = sc.integrate.odeint(EGFRkineticsGompertz_opt2_Pharmacokinetics, h0_local, intTime, args=(Vlimit, r_sen, r_res, r_per, p_mutation, beta, doseinterval))
        cellsTreat_dist = sc.integrate.odeint(EGFRkineticsGompertz_opt2_Pharmacokinetics, h0_dist, intTime, args=(Vlimit, r_sen, r_res, r_per, p_mutation, beta, doseinterval))

        # Extract number at the end of day
        # Append result
        cellSensTreated[step+1:step+10,:]=np.transpose(np.array([(cellsTreat_local[1:10,0])+(cellsTreat_dist[1:10,0]), (cellsTreat_local[1:10,0]), (cellsTreat_dist[1:10,0])]))
        cellResTreated[step+1:step+10,:]= np.transpose(np.array([(cellsTreat_local[1:10,1])+(cellsTreat_dist[1:10,1]), (cellsTreat_local[1:10,1]), (cellsTreat_dist[1:10,1])]))
        cellPerTreated[step+1:step+10,:]= np.transpose(np.array([(cellsTreat_local[1:10,2])+(cellsTreat_dist[1:10,2]), (cellsTreat_local[1:10,2]), (cellsTreat_dist[1:10,2])]))
        erlotinibPlasma[step+1:step+10]=cellsTreat_local[1:10,3]

        #if a comparantent has <1 cell set compartment to 0
        cellSensTreated[step:step+10,cellSensTreated[step+9,:]<1]=0
        cellResTreated[step:step+10,cellResTreated[step+9,:]<1]=0
        cellPerTreated[step:step+10,cellPerTreated[step+9,:]<1]=0
        
        # Update initial condition for next step
        h0_local =  np.array([cellSensTreated[step+9,1], cellResTreated[step+9,1], cellPerTreated[step+9,1],erlotinibPlasma[step+9]])
        h0_dist =   np.array([cellSensTreated[step+9,2], cellResTreated[step+9,2], cellPerTreated[step+9,2],erlotinibPlasma[step+9]])
        h0 = h0_local+h0_dist; h0[3] = h0[3]/2; # because dose at index doubled when local and distant addded           
        
        cellTotalTreated=cellSensTreated+cellResTreated+cellPerTreated
        if cellTotalTreated[step+9,0]<1:
            Cured =True
            t_cure = step
            break
        
        if ((cellTotalTreated[step+9,1]>N0_total)|(cellTotalTreated[step+9,2]>G_DRRlvl)) & (step>10) & (np.mod(step,420)==0):
            break        

    ####### regrowth after treatment if not cured or progression #######    
    steps_remaining = 18010-(step)
    if (Cured==False)&(steps_remaining>0): 
        intTime = np.arange(0,steps_remaining,1)
        cellsGrow_local = sc.integrate.odeint(GompertzMutation, h0_local[0:3], intTime, args=(Vlimit, r_sen/10, r_res/10, r_sen/10, p_mutation))        
        cellsGrow_dist  = sc.integrate.odeint(GompertzMutation, h0_dist[0:3], intTime, args=(Vlimit, r_sen/10, r_res/10, r_sen/10, p_mutation))        
        cellSensTreated[step+1:18010,:]=np.transpose(np.array([(cellsGrow_local[1:,0])+(cellsGrow_dist[1:,0]), (cellsGrow_local[1:,0]), (cellsGrow_dist[1:,0])]))
        cellResTreated[step+1:18010,:]= np.transpose(np.array([(cellsGrow_local[1:,1])+(cellsGrow_dist[1:,1]), (cellsGrow_local[1:,1]), (cellsGrow_dist[1:,1])]))
        cellPerTreated[step+1:18010,:]= np.transpose(np.array([(cellsGrow_local[1:,2])+(cellsGrow_dist[1:,2]), (cellsGrow_local[1:,2]), (cellsGrow_dist[1:,2])]))
        cellTotalTreated[step+1:18010,:] = cellSensTreated[step+1:18010,:]+cellResTreated[step+1:18010,:]+cellPerTreated[step+1:18010,:]

    ####### Comorbid Sample Russian Roullete #######    
    comorbidRate = 1-(.288/(.113+.288))**(1/60) #0.0055 mo^{-1}
    r = np.random.uniform(0,1,1)
    T_Mo_comorbid = (np.log(-(r-1))/np.log(1-comorbidRate))
        
    ####### find when patient dies and tumor progression ####### 
    # death or cure cancer
    if cellTotalTreated[18000,0]> EndT:
        ind_Tdeath=next(i for i,v in enumerate(cellTotalTreated[:,0]) if v > EndT)
        T_Mo_death=ind_Tdeath/10/30 
        T_Mo_cure = 60    
    elif Cured:
        T_Mo_cure = t_cure/10/30
        T_Mo_death = 60
    else:
        T_Mo_death = 60
        T_Mo_cure = 60
    
    # compare death or cure to randomly sampled comorbid
    if (T_Mo_comorbid<60) & (T_Mo_comorbid<T_Mo_death):
        T_Mo_death = T_Mo_comorbid
        T_Mo_cure = 60
           
    #calc min vol & time (every 6 wks per EURTAC trial)
    VminDelt_3mo_tot = (np.min(cellTotalTreated[420:18010:420,0])-cellTotalTreated[0,0])/cellTotalTreated[0,0]     

    # find local and distant failures (every 6 wks per EURTAC trial, 17640 is the last multiple of 420 less than 18000)
    if cellTotalTreated[17640,1]> N0_total:
        ind_LRR_3mo=next(i for i,v in enumerate(cellTotalTreated[420:18010:420,1]) if v > N0_total)  
        T_Mo_LRR_3mo = (ind_LRR_3mo*420+420)/10/30    
    else:
        T_Mo_LRR_3mo = 60        
    if cellTotalTreated[17640,2]> G_DRRlvl:
        ind_DRR_3mo=next(i for i,v in enumerate(cellTotalTreated[420:18010:420,2]) if v > G_DRRlvl)  
        T_Mo_DRR_3mo = (ind_DRR_3mo*420+420)/10/30    
    else:
        T_Mo_DRR_3mo = 60        
    
    T_prog = np.min([T_Mo_death,T_Mo_LRR_3mo,T_Mo_DRR_3mo])          

    out_PR = np.zeros(4)
    out_PR[0] = T_Mo_LRR_3mo; out_PR[1] = T_Mo_DRR_3mo; out_PR[2] = T_prog;out_PR[3] = VminDelt_3mo_tot;
    
    return out_PR

In [9]:
####-------------------------------------------------------------------------------------------------------####
#
# TKI pre CRT
#
####-------------------------------------------------------------------------------------------------------####

def Dynamic_TKIpreCRT_MetsDRRLRR_CoMorbid_local(i):

    # select index of model parameter (i.e. define patient)
    N0_total=GN0_total[i]
    N0_persister_frac=GN0_persister_frac[i]
    N0_resistance=GN0_resistance[i]
    r_sen=Gr_sen[i]
    r_per=Gr_per[i]
    r_res=Gr_res[i]
    beta=Gbeta[i]
    wkTKIset = G_wkTKIset[i] 
    
    h0=np.array([N0_total*(1-N0_persister_frac)-N0_resistance,N0_resistance,N0_total*N0_persister_frac])
    p_mutation = G_mu_set
    
    h0_local = h0
    h0_dist = (Gmet_scl)*h0
    h0 = (1+Gmet_scl)*h0
        
    # Lists to store results
    # total,local,distant
    cellSensTreated = np.zeros([18010,3]); #(30*12*5+1)*10
    cellPerTreated = np.zeros([18010,3]);
    cellResTreated = np.zeros([18010,3]);
    cellTotalTreated = np.zeros([18010,3]);    
    
    cellSensTreated[0,0] = h0[0];
    cellPerTreated[0,0] =  h0[2];
    cellResTreated[0,0] =  h0[1];
    cellTotalTreated[0,0] = np.sum(h0); 
    
    cellSensTreated[0,1] = h0_local[0];
    cellPerTreated[0,1] =  h0_local[2];
    cellResTreated[0,1] =  h0_local[1];
    cellTotalTreated[0,1] = np.sum(h0_local); 
    
    cellSensTreated[0,2] = h0_dist[0];
    cellPerTreated[0,2] =  h0_dist[2];
    cellResTreated[0,2] =  h0_dist[1];
    cellTotalTreated[0,2] = np.sum(h0_dist); 
    
    ####### TKI wkTKIset weeks #######
    
    # Vector of initial conditions
    y0 = 0
    doseErlotinibDaily = 150.0;
    # get plasma level for every timepoint
    
    #t definition of TIME
    # set up TKI daily dosing for pharmacokinetics 
    tInterval = 0.1
    tTKI = np.arange(tInterval, wkTKIset*7+tInterval/10, tInterval)
    daysTKI = wkTKIset*7  # Number of days to simulate 
    erlotinibStart = 0
    doseErlotinibDaily = 150.0;
    doseIntervalDaily = np.zeros(np.size(tTKI))+1;
    plasmadoseIntervalDaily = np.zeros(np.size(tTKI)+1);
    for tCounter in np.arange(erlotinibStart*10, (daysTKI)*10, 10):  
            doseIntervalDaily[int(tCounter)]=doseErlotinibDaily
            plasmadoseIntervalDaily[int(tCounter)]= 1.0182 + 0.0114*doseErlotinibDaily;    
    erlotinibPlasma=np.zeros(np.size(tTKI)+1)
    erlotinibPlasma[0]=y0
    
    # TKI Treatment Loop 
    step=0;Cured=False;t_cure = 18000;
    for step in range(1,np.size(tTKI)+1):
        intTime = [0, 1]
        # Get dose for this step
        
        # Solve the pharmacokinetic ODEs
        erlotinib = sc.integrate.odeint(Pharmacokinetics, y0, intTime, args=(plasmadoseIntervalDaily[step], ))
        # Append result
        erlotinibPlasma[step]=erlotinib[0]
        # Update initial condition for next step
        y0 = [ float(erlotinib[1]), ]

        # Get dose for this step
        drugLevel = erlotinibPlasma[step]   
        
        # Solve the TKI ODEs
        cellsTreat_local = sc.integrate.odeint(EGFRkineticsGompertz_opt2, h0_local, intTime, args=(Vlimit, r_sen/10, r_res/10, r_per/10, p_mutation, beta,drugLevel))
        cellsTreat_dist = sc.integrate.odeint(EGFRkineticsGompertz_opt2, h0_dist, intTime, args=(Vlimit, r_sen/10, r_res/10, r_per/10, p_mutation, beta,drugLevel))

        # Extract number at the end of day
        # Append result
        cellSensTreated[step,:]=np.array([float(cellsTreat_local[1,0])+float(cellsTreat_dist[1,0]), float(cellsTreat_local[1,0]), float(cellsTreat_dist[1,0])])
        cellResTreated[step,:]= np.array([float(cellsTreat_local[1,1])+float(cellsTreat_dist[1,1]), float(cellsTreat_local[1,1]), float(cellsTreat_dist[1,1])])
        cellPerTreated[step,:]= np.array([float(cellsTreat_local[1,2])+float(cellsTreat_dist[1,2]), float(cellsTreat_local[1,2]), float(cellsTreat_dist[1,2])])

        #if a comparantent has <1 cell set compartment to 0
        cellSensTreated[step,cellSensTreated[step,:]<1]=0
        cellResTreated[step,cellResTreated[step,:]<1]=0
        cellPerTreated[step,cellPerTreated[step,:]<1]=0
        
        # Update initial condition for next step
        h0_local =  np.array([cellSensTreated[step,1], cellResTreated[step,1], cellPerTreated[step,1]])
        h0_dist =   np.array([cellSensTreated[step,2], cellResTreated[step,2], cellPerTreated[step,2]])
        h0 = h0_local+h0_dist
        
        cellTotalTreated[step,:]=cellSensTreated[step,:]+cellResTreated[step,:]+cellPerTreated[step,:]
        if cellTotalTreated[step,0]<1:
            Cured =True
            t_cure = step
            break
        
    ####### CRT 7 weeks #######
    
    Vinblastine_C0=5
    Cisplatin_C0=100
    current_alpha=Galpha[i]
    t_half=1
    
    next_step = np.size(tTKI)+1
    tCRT = np.arange(wkTKIset*7+tInterval,wkTKIset*7+7*7+tInterval/10, tInterval)
    days = np.round(np.arange(1.,50,tInterval),1)

    #only local in field cells affected by CRT
    #day will count from start of treatment
    if Cured==False:
        for step in range(next_step,next_step+np.size(tCRT)): 
            #5 weeks vinblastin +cisplatin
            if days[step-next_step]<36 and np.mod(days[step-next_step]+6,7)==0:
                Vinblastine_C=Vinblastine_C0;
                h0_local=h0_local*chemo(GVinblastine_u[i],Vinblastine_C);
                h0_dist=h0_dist*chemo(GVinblastine_u[i],Vinblastine_C);
            elif days[step-next_step]<36 and np.mod(days[step-next_step],1)==0:
                Vinblastine_C=Vinblastine_C0*(1./2)**((np.mod(days[step-next_step]-1,7))/t_half)  #decay
                h0_local=h0_local*chemo(GVinblastine_u[i],Vinblastine_C);
                h0_dist=h0_dist*chemo(GVinblastine_u[i],Vinblastine_C);
            if days[step-next_step]==1 or days[step-next_step]==29:
                Cisplatin_C=Cisplatin_C0;
                h0_local=h0_local*chemo(GCisplatin_u[i],Cisplatin_C);
                h0_dist=h0_dist*chemo(GCisplatin_u[i],Cisplatin_C);
            elif days[step-next_step]<50 and np.mod(days[step-next_step],1)==0:
                Cisplatin_C=Cisplatin_C0*(1./2)**((np.mod(days[step-next_step]-1,28))/t_half)  #decay
                h0_local=h0_local*chemo(GCisplatin_u[i],Cisplatin_C);
                h0_dist=h0_dist*chemo(GCisplatin_u[i],Cisplatin_C);
            
            # 60 Gy  2*30 - local only
            if (days[step-next_step]<42):
                if ((np.mod(days[step-next_step]+6,7)==0 or np.mod(days[step-next_step]+5,7)==0 or np.mod(days[step-next_step]+4,7)==0 or np.mod(days[step-next_step]+3,7)==0 or np.mod(days[step-next_step]+2,7)==0)):  
                    h0_local=h0_local*radiation(current_alpha,current_alpha/10,2)
         
            #Make New Gompertz Growth +Mutation 
            intTime = [0, 1]
            cellsGrow_local = sc.integrate.odeint(GompertzMutation, h0_local, intTime, args=(Vlimit, r_sen/10, r_res/10, r_sen/10, p_mutation))        
            cellsGrow_dist  = sc.integrate.odeint(GompertzMutation, h0_dist, intTime, args=(Vlimit, r_sen/10, r_res/10, r_sen/10, p_mutation))        

            # Extract number at the end of day
            # Append result
            cellSensTreated[step,:]=np.array([float(cellsGrow_local[1,0])+float(cellsGrow_dist[1,0]), float(cellsGrow_local[1,0]), float(cellsGrow_dist[1,0])])
            cellResTreated[step,:]= np.array([float(cellsGrow_local[1,1])+float(cellsGrow_dist[1,1]), float(cellsGrow_local[1,1]), float(cellsGrow_dist[1,1])])
            cellPerTreated[step,:]= np.array([float(cellsGrow_local[1,2])+float(cellsGrow_dist[1,2]), float(cellsGrow_local[1,2]), float(cellsGrow_dist[1,2])])

            #if a comparantent has <1 cell set compartment to 0
            cellSensTreated[step,cellSensTreated[step,:]<1]=0
            cellResTreated[step,cellResTreated[step,:]<1]=0
            cellPerTreated[step,cellPerTreated[step,:]<1]=0
            
            # Update initial condition for next step
            h0_local =  np.array([cellSensTreated[step,1], cellResTreated[step,1], cellPerTreated[step,1]])
            h0_dist =   np.array([cellSensTreated[step,2], cellResTreated[step,2], cellPerTreated[step,2]])
            h0 = h0_local+h0_dist
            
            cellTotalTreated[step,:]=cellSensTreated[step,:]+cellResTreated[step,:]+cellPerTreated[step,:]
            if cellTotalTreated[step,0]<1:
                Cured =True
                t_cure = step
                break
                
    ####### regrowth after treatment if not cured or progression #######    
    steps_remaining = 18010-(step)
    if (Cured==False): 
        intTime = np.arange(0,steps_remaining,1)
        cellsGrow_local = sc.integrate.odeint(GompertzMutation, h0_local, intTime, args=(Vlimit, r_sen/10, r_res/10, r_sen/10, p_mutation))        
        cellsGrow_dist  = sc.integrate.odeint(GompertzMutation, h0_dist, intTime, args=(Vlimit, r_sen/10, r_res/10, r_sen/10, p_mutation))        
        cellSensTreated[step+1:18010,:]=np.transpose(np.array([(cellsGrow_local[1:,0])+(cellsGrow_dist[1:,0]), (cellsGrow_local[1:,0]), (cellsGrow_dist[1:,0])]))
        cellResTreated[step+1:18010,:]= np.transpose(np.array([(cellsGrow_local[1:,1])+(cellsGrow_dist[1:,1]), (cellsGrow_local[1:,1]), (cellsGrow_dist[1:,1])]))
        cellPerTreated[step+1:18010,:]= np.transpose(np.array([(cellsGrow_local[1:,2])+(cellsGrow_dist[1:,2]), (cellsGrow_local[1:,2]), (cellsGrow_dist[1:,2])]))
        cellTotalTreated[step+1:18010,:] = cellSensTreated[step+1:18010,:]+cellResTreated[step+1:18010,:]+cellPerTreated[step+1:18010,:]

    ####### Comorbid Sample Russian Roullete #######    
    comorbidRate = 1-(.288/(.113+.288))**(1/60) #0.0055 mo^{-1}
    r = np.random.uniform(0,1,1)
    T_Mo_comorbid = (np.log(-(r-1))/np.log(1-comorbidRate))
        
    ####### find when patient dies and tumor progression ####### 
    # death or cure cancer
    if cellTotalTreated[18000,0]> EndT:
        ind_Tdeath=next(i for i,v in enumerate(cellTotalTreated[:,0]) if v > EndT)
        T_Mo_death = ind_Tdeath/10/30
        T_Mo_cure = 60    
    elif Cured:
        T_Mo_cure = t_cure/10/30
        T_Mo_death = 60
    else:
        T_Mo_death = 60
        T_Mo_cure = 60
    
    # compare death or cure to randomly sampled comorbid
    if (T_Mo_comorbid<60) & (T_Mo_comorbid<T_Mo_death):
        T_Mo_death = T_Mo_comorbid
        T_Mo_cure = 60
           
    # find local and distant failures every 3 mo.
    if cellTotalTreated[18000,1]> N0_total:
        ind_LRR_3mo=next(i for i,v in enumerate(cellTotalTreated[900:18010:900,1]) if v > N0_total)  
        T_Mo_LRR_3mo = (ind_LRR_3mo*900+900)/10/30    
    else:
        T_Mo_LRR_3mo = 60        
    if cellTotalTreated[18000,2]> G_DRRlvl:
        ind_DRR_3mo=next(i for i,v in enumerate(cellTotalTreated[900:18010:900,2]) if v > G_DRRlvl)  
        T_Mo_DRR_3mo = (ind_DRR_3mo*900+900)/10/30    
    else:
        T_Mo_DRR_3mo = 60        
    
    T_prog = np.min([T_Mo_death,T_Mo_LRR_3mo,T_Mo_DRR_3mo])          

    out_PR = np.zeros(3)
    out_PR[0] = T_Mo_LRR_3mo; out_PR[1] = T_Mo_DRR_3mo; out_PR[2] = T_prog;
    
    return out_PR

In [10]:
####-------------------------------------------------------------------------------------------------------####
#
# TKI pre CRT -- Sequential
#
####-------------------------------------------------------------------------------------------------------####

def Dynamic_TKIpreCRT_Sequential_MetsDRRLRR_CoMorbid_local(i):

    # select index of model parameter (i.e. define patient)
    N0_total=GN0_total[i]
    N0_persister_frac=GN0_persister_frac[i]
    N0_resistance=GN0_resistance[i]
    r_sen=Gr_sen[i]
    r_per=Gr_per[i]
    r_res=Gr_res[i]
    beta=Gbeta[i]
    wkTKIset = G_wkTKIset[i] 
    
    h0=np.array([N0_total*(1-N0_persister_frac)-N0_resistance,N0_resistance,N0_total*N0_persister_frac])
    p_mutation = G_mu_set
    
    h0_local = h0
    h0_dist = (Gmet_scl)*h0
    h0 = (1+Gmet_scl)*h0
        
    # Lists to store results
    # total,local,distant
    cellSensTreated = np.zeros([18010,3]); #(30*12*5+1)*10
    cellPerTreated = np.zeros([18010,3]);
    cellResTreated = np.zeros([18010,3]);
    cellTotalTreated = np.zeros([18010,3]);    
    
    cellSensTreated[0,0] = h0[0];
    cellPerTreated[0,0] =  h0[2];
    cellResTreated[0,0] =  h0[1];
    cellTotalTreated[0,0] = np.sum(h0); 
    
    cellSensTreated[0,1] = h0_local[0];
    cellPerTreated[0,1] =  h0_local[2];
    cellResTreated[0,1] =  h0_local[1];
    cellTotalTreated[0,1] = np.sum(h0_local); 
    
    cellSensTreated[0,2] = h0_dist[0];
    cellPerTreated[0,2] =  h0_dist[2];
    cellResTreated[0,2] =  h0_dist[1];
    cellTotalTreated[0,2] = np.sum(h0_dist); 
    
    ####### TKI wkTKIset weeks #######
    
    # Vector of initial conditions
    y0 = 0
    doseErlotinibDaily = 150.0;
    # get plasma level for every timepoint

    #t definition of TIME
    # set up TKI daily dosing for pharmacokinetics 
    tInterval = 0.1
    tTKI = np.arange(tInterval, wkTKIset*7+tInterval/10, tInterval)
    daysTKI = wkTKIset*7  # Number of days to simulate 
    erlotinibStart = 0
    doseErlotinibDaily = 150.0;
    doseIntervalDaily = np.zeros(np.size(tTKI))+1;
    plasmadoseIntervalDaily = np.zeros(np.size(tTKI)+1);
    for tCounter in np.arange(erlotinibStart*10, (daysTKI)*10, 10): 
            doseIntervalDaily[int(tCounter)]=doseErlotinibDaily
            plasmadoseIntervalDaily[int(tCounter)]= 1.0182 + 0.0114*doseErlotinibDaily;    

    erlotinibPlasma=np.zeros(np.size(tTKI)+1)
    erlotinibPlasma[0]=y0
    
    # TKI Treatment Loop 
    step=0;Cured=False;t_cure = 18000;
    for step in range(1,np.size(tTKI)+1):
        intTime = [0, 1]
        # Get dose for this step
        
        # Solve the pharmacokinetics ODE
        erlotinib =  sc.integrate.odeint(Pharmacokinetics, y0, intTime, args=(plasmadoseIntervalDaily[step], ))
        # Append result
        erlotinibPlasma[step]=erlotinib[0]
        # Update initial condition for next step
        y0 = [ float(erlotinib[1]), ]

        # Get dose for this step
        drugLevel = erlotinibPlasma[step]   
        
        # Solve the TKI ODEs
        cellsTreat_local = sc.integrate.odeint(EGFRkineticsGompertz_opt2, h0_local, intTime, args=(Vlimit, r_sen/10, r_res/10, r_per/10, p_mutation, beta,drugLevel))
        cellsTreat_dist = sc.integrate.odeint(EGFRkineticsGompertz_opt2, h0_dist, intTime, args=(Vlimit, r_sen/10, r_res/10, r_per/10, p_mutation, beta,drugLevel))

        # Extract number at the end of day
        # Append result
        cellSensTreated[step,:]=np.array([float(cellsTreat_local[1,0])+float(cellsTreat_dist[1,0]), float(cellsTreat_local[1,0]), float(cellsTreat_dist[1,0])])
        cellResTreated[step,:]= np.array([float(cellsTreat_local[1,1])+float(cellsTreat_dist[1,1]), float(cellsTreat_local[1,1]), float(cellsTreat_dist[1,1])])
        cellPerTreated[step,:]= np.array([float(cellsTreat_local[1,2])+float(cellsTreat_dist[1,2]), float(cellsTreat_local[1,2]), float(cellsTreat_dist[1,2])])

        #if a comparantent has <1 cell set compartment to 0
        cellSensTreated[step,cellSensTreated[step,:]<1]=0
        cellResTreated[step,cellResTreated[step,:]<1]=0
        cellPerTreated[step,cellPerTreated[step,:]<1]=0
        
        # Update initial condition for next step
        h0_local =  np.array([cellSensTreated[step,1], cellResTreated[step,1], cellPerTreated[step,1]])
        h0_dist =   np.array([cellSensTreated[step,2], cellResTreated[step,2], cellPerTreated[step,2]])
        h0 = h0_local+h0_dist
        
        cellTotalTreated[step,:]=cellSensTreated[step,:]+cellResTreated[step,:]+cellPerTreated[step,:]
        if cellTotalTreated[step,0]<1:
            Cured =True
            t_cure = step
            break
        
    ####### Sequential CRT 7 weeks + 60 Gy RT on day 50 #######
    
    Vinblastine_C0=5
    Cisplatin_C0=100
    current_alpha=Galpha[i]
    t_half=1
    
    next_step = np.size(tTKI)+1
    tCRT = np.arange(wkTKIset*7+tInterval,wkTKIset*7+7*7+45+tInterval/10, tInterval)
    days = np.round(np.arange(1.,50+45,tInterval),1)

    #only local in field cells affected by CRT
    #day will count from start of treatment
    if Cured==False:
        for step in range(next_step,next_step+np.size(tCRT)): 
            if days[step-next_step]<50:
                #5 weeks vinblastin +cisplatin
                if days[step-next_step]<36 and np.mod(days[step-next_step]+6,7)==0:
                    Vinblastine_C=Vinblastine_C0;
                    h0_local=h0_local*chemo(GVinblastine_u[i],Vinblastine_C);
                    h0_dist=h0_dist*chemo(GVinblastine_u[i],Vinblastine_C);
                elif days[step-next_step]<36 and np.mod(days[step-next_step],1)==0:
                    Vinblastine_C=Vinblastine_C0*(1./2)**((np.mod(days[step-next_step]-1,7))/t_half)  #decay
                    h0_local=h0_local*chemo(GVinblastine_u[i],Vinblastine_C);
                    h0_dist=h0_dist*chemo(GVinblastine_u[i],Vinblastine_C);
                if days[step-next_step]==1 or days[step-next_step]==29:
                    Cisplatin_C=Cisplatin_C0;
                    h0_local=h0_local*chemo(GCisplatin_u[i],Cisplatin_C);
                    h0_dist=h0_dist*chemo(GCisplatin_u[i],Cisplatin_C);
                elif days[step-next_step]<50 and np.mod(days[step-next_step],1)==0:
                    Cisplatin_C=Cisplatin_C0*(1./2)**((np.mod(days[step-next_step]-1,28))/t_half)  #decay
                    h0_local=h0_local*chemo(GCisplatin_u[i],Cisplatin_C);
                    h0_dist=h0_dist*chemo(GCisplatin_u[i],Cisplatin_C);
            else:
                # 60 Gy  2*30 - local only
                if (days[step-next_step]-49<42):
                    if ((np.mod(days[step-next_step]+6,7)==0 or np.mod(days[step-next_step]+5,7)==0 or np.mod(days[step-next_step]+4,7)==0 or np.mod(days[step-next_step]+3,7)==0 or np.mod(days[step-next_step]+2,7)==0)):  
                        h0_local=h0_local*radiation(current_alpha,current_alpha/10,2)

                        
            #Make New Gompertz Growth + Mutation 
            intTime = [0, 1]
            cellsGrow_local = sc.integrate.odeint(GompertzMutation, h0_local, intTime, args=(Vlimit, r_sen/10, r_res/10, r_sen/10, p_mutation))        
            cellsGrow_dist  = sc.integrate.odeint(GompertzMutation, h0_dist, intTime, args=(Vlimit, r_sen/10, r_res/10, r_sen/10, p_mutation))        

            # Extract number at the end of day
            # Append result
            cellSensTreated[step,:]=np.array([float(cellsGrow_local[1,0])+float(cellsGrow_dist[1,0]), float(cellsGrow_local[1,0]), float(cellsGrow_dist[1,0])])
            cellResTreated[step,:]= np.array([float(cellsGrow_local[1,1])+float(cellsGrow_dist[1,1]), float(cellsGrow_local[1,1]), float(cellsGrow_dist[1,1])])
            cellPerTreated[step,:]= np.array([float(cellsGrow_local[1,2])+float(cellsGrow_dist[1,2]), float(cellsGrow_local[1,2]), float(cellsGrow_dist[1,2])])

            #if a comparantent has <1 cell set compartment to 0
            cellSensTreated[step,cellSensTreated[step,:]<1]=0
            cellResTreated[step,cellResTreated[step,:]<1]=0
            cellPerTreated[step,cellPerTreated[step,:]<1]=0
            
            # Update initial condition for next step
            h0_local =  np.array([cellSensTreated[step,1], cellResTreated[step,1], cellPerTreated[step,1]])
            h0_dist =   np.array([cellSensTreated[step,2], cellResTreated[step,2], cellPerTreated[step,2]])
            h0 = h0_local+h0_dist
            
            cellTotalTreated[step,:]=cellSensTreated[step,:]+cellResTreated[step,:]+cellPerTreated[step,:]
            if cellTotalTreated[step,0]<1:
                Cured =True
                t_cure = step
                break
                
    ####### regrowth after treatment if not cured or progression #######    
    steps_remaining = 18010-(step)
    if (Cured==False): 
        intTime = np.arange(0,steps_remaining,1)
        cellsGrow_local = sc.integrate.odeint(GompertzMutation, h0_local, intTime, args=(Vlimit, r_sen/10, r_res/10, r_sen/10, p_mutation))        
        cellsGrow_dist  = sc.integrate.odeint(GompertzMutation, h0_dist, intTime, args=(Vlimit, r_sen/10, r_res/10, r_sen/10, p_mutation))        
        cellSensTreated[step+1:18010,:]=np.transpose(np.array([(cellsGrow_local[1:,0])+(cellsGrow_dist[1:,0]), (cellsGrow_local[1:,0]), (cellsGrow_dist[1:,0])]))
        cellResTreated[step+1:18010,:]= np.transpose(np.array([(cellsGrow_local[1:,1])+(cellsGrow_dist[1:,1]), (cellsGrow_local[1:,1]), (cellsGrow_dist[1:,1])]))
        cellPerTreated[step+1:18010,:]= np.transpose(np.array([(cellsGrow_local[1:,2])+(cellsGrow_dist[1:,2]), (cellsGrow_local[1:,2]), (cellsGrow_dist[1:,2])]))
        cellTotalTreated[step+1:18010,:] = cellSensTreated[step+1:18010,:]+cellResTreated[step+1:18010,:]+cellPerTreated[step+1:18010,:]

    ####### Comorbid Sample Russian Roullete #######    
    comorbidRate = 1-(.288/(.113+.288))**(1/60) #0.0055 mo^{-1}
    r = np.random.uniform(0,1,1)
    T_Mo_comorbid = (np.log(-(r-1))/np.log(1-comorbidRate))
        
    ####### find when patient dies and tumor progression ####### 
    # death or cure cancer
    if cellTotalTreated[18000,0]> EndT:
        ind_Tdeath=next(i for i,v in enumerate(cellTotalTreated[:,0]) if v > EndT)
        T_Mo_death = ind_Tdeath/10/30
        T_Mo_cure = 60    
    elif Cured:
        T_Mo_cure = t_cure/10/30
        T_Mo_death = 60
    else:
        T_Mo_death = 60
        T_Mo_cure = 60
    
    # compare death or cure to randomly sampled comorbid
    if (T_Mo_comorbid<60) & (T_Mo_comorbid<T_Mo_death):
        T_Mo_death = T_Mo_comorbid
        T_Mo_cure = 60
           
    # find local and distant failures every 3 mo.
    if cellTotalTreated[18000,1]> N0_total:
        ind_LRR_3mo=next(i for i,v in enumerate(cellTotalTreated[900:18010:900,1]) if v > N0_total)  
        T_Mo_LRR_3mo = (ind_LRR_3mo*900+900)/10/30    
    else:
        T_Mo_LRR_3mo = 60        
    if cellTotalTreated[18000,2]> G_DRRlvl:
        ind_DRR_3mo=next(i for i,v in enumerate(cellTotalTreated[900:18010:900,2]) if v > G_DRRlvl)  
        T_Mo_DRR_3mo = (ind_DRR_3mo*900+900)/10/30    
    else:
        T_Mo_DRR_3mo = 60        
    
    T_prog = np.min([T_Mo_death,T_Mo_LRR_3mo,T_Mo_DRR_3mo])          

    out_PR = np.zeros(3)
    out_PR[0] = T_Mo_LRR_3mo; out_PR[1] = T_Mo_DRR_3mo; out_PR[2] = T_prog;
    
    return out_PR

In [11]:
####-------------------------------------------------------------------------------------------------------####
#
# TKI pre and post CRT
#
####-------------------------------------------------------------------------------------------------------####

def Dynamic_TKIpreANDpostCRTuntilProg_MetsDRRLRR_CoMorbid_local(i):
    
    # select index of model parameter (i.e. define patient)
    N0_total=GN0_total[i]
    N0_persister_frac=GN0_persister_frac[i]
    N0_resistance=GN0_resistance[i]
    r_sen=Gr_sen[i]
    r_per=Gr_per[i]
    r_res=Gr_res[i]
    beta=Gbeta[i]
    wkTKIset = G_wkTKIset[i] 
    
    h0=np.array([N0_total*(1-N0_persister_frac)-N0_resistance,N0_resistance,N0_total*N0_persister_frac])
    p_mutation = G_mu_set
    
    h0_local = h0
    h0_dist = (Gmet_scl)*h0
    h0 = (1+Gmet_scl)*h0
        
    # Arrays to store results
    # total,local,distant
    cellSensTreated = np.zeros([18010,3]); #(30*12*5+1)*10
    cellPerTreated = np.zeros([18010,3]);
    cellResTreated = np.zeros([18010,3]);
    cellTotalTreated = np.zeros([18010,3]);    
    erlotinibPlasma = np.zeros(18010); 

    cellSensTreated[0,0] = h0[0];
    cellPerTreated[0,0] =  h0[2];
    cellResTreated[0,0] =  h0[1];
    cellTotalTreated[0,0] = np.sum(h0); 
    
    cellSensTreated[0,1] = h0_local[0];
    cellPerTreated[0,1] =  h0_local[2];
    cellResTreated[0,1] =  h0_local[1];
    cellTotalTreated[0,1] = np.sum(h0_local); 
    
    cellSensTreated[0,2] = h0_dist[0];
    cellPerTreated[0,2] =  h0_dist[2];
    cellResTreated[0,2] =  h0_dist[1];
    cellTotalTreated[0,2] = np.sum(h0_dist); 
    
    ####### TKI wkTKIset weeks #######
    
    # Vector of initial conditions
    y0 = 0
    doseErlotinibDaily = 150.0;
    # get plasma level for every timepoint

    #t definition of TIME
    # set up TKI daily dosing for pharmacokinetics 
    tInterval = 0.1
    tTKI = np.arange(tInterval, wkTKIset*7+tInterval/10, tInterval)
    daysTKI = wkTKIset*7  # Number of days to simulate
    erlotinibStart = 0
    doseErlotinibDaily = 150.0;
    doseIntervalDaily = np.zeros(np.size(tTKI))+1;
    plasmadoseIntervalDaily = np.zeros(np.size(tTKI)+1);
    for tCounter in np.arange(erlotinibStart*10, (daysTKI)*10, 10):  
            doseIntervalDaily[int(tCounter)]=doseErlotinibDaily
            plasmadoseIntervalDaily[int(tCounter)]= 1.0182 + 0.0114*doseErlotinibDaily;    
    erlotinibPlasma[0]=y0    
    
    # TKI Treatment Loop 
    step=0;Cured=False;t_cure = 18000;
    for step in range(1,np.size(tTKI)+1):
        intTime = [0, 1]
        # Get dose for this step
        
        # Solve the pharmacokinetic ODEs
        erlotinib =  sc.integrate.odeint(Pharmacokinetics, y0, intTime, args=(plasmadoseIntervalDaily[step], ))
        # Append result
        erlotinibPlasma[step]=erlotinib[0]
        # Update initial condition for next step
        y0 = [ float(erlotinib[1]), ]

        # Get dose for this step
        drugLevel = erlotinibPlasma[step]   
        
        # Solve the TKI ODEs
        cellsTreat_local = sc.integrate.odeint(EGFRkineticsGompertz_opt2, h0_local, intTime, args=(Vlimit, r_sen/10, r_res/10, r_per/10, p_mutation, beta,drugLevel))
        cellsTreat_dist = sc.integrate.odeint(EGFRkineticsGompertz_opt2, h0_dist, intTime, args=(Vlimit, r_sen/10, r_res/10, r_per/10, p_mutation, beta,drugLevel))

        # Extract number at the end of day
        # Append result
        cellSensTreated[step,:]=np.array([float(cellsTreat_local[1,0])+float(cellsTreat_dist[1,0]), float(cellsTreat_local[1,0]), float(cellsTreat_dist[1,0])])
        cellResTreated[step,:]= np.array([float(cellsTreat_local[1,1])+float(cellsTreat_dist[1,1]), float(cellsTreat_local[1,1]), float(cellsTreat_dist[1,1])])
        cellPerTreated[step,:]= np.array([float(cellsTreat_local[1,2])+float(cellsTreat_dist[1,2]), float(cellsTreat_local[1,2]), float(cellsTreat_dist[1,2])])

        #if a comparantent has <1 cell set compartment to 0
        cellSensTreated[step,cellSensTreated[step,:]<1]=0
        cellResTreated[step,cellResTreated[step,:]<1]=0
        cellPerTreated[step,cellPerTreated[step,:]<1]=0
        
        # Update initial condition for next step
        h0_local =  np.array([cellSensTreated[step,1], cellResTreated[step,1], cellPerTreated[step,1]])
        h0_dist =   np.array([cellSensTreated[step,2], cellResTreated[step,2], cellPerTreated[step,2]])
        h0 = h0_local+h0_dist
        
        cellTotalTreated[step,:]=cellSensTreated[step,:]+cellResTreated[step,:]+cellPerTreated[step,:]
        if cellTotalTreated[step,0]<1:
            Cured =True
            t_cure = step
            break
        
    ####### CRT 7 weeks #######
    Vinblastine_C0=5
    Cisplatin_C0=100
    current_alpha=Galpha[i]
    t_half=1
    
    next_step = np.size(tTKI)+1
    tCRT = np.arange(wkTKIset*7+tInterval,wkTKIset*7+7*7+tInterval/10, tInterval)
    days = np.round(np.arange(1.,50,tInterval),1)

    #only local in field cells affected by CRT
    #day will count from start of treatment
    if Cured==False:
        for step in range(next_step,next_step+np.size(tCRT)): 
            #5 weeks vinblastin + cisplatin
            if days[step-next_step]<36 and np.mod(days[step-next_step]+6,7)==0:
                Vinblastine_C=Vinblastine_C0;
                h0_local=h0_local*chemo(GVinblastine_u[i],Vinblastine_C);
                h0_dist=h0_dist*chemo(GVinblastine_u[i],Vinblastine_C);
            elif days[step-next_step]<36 and np.mod(days[step-next_step],1)==0:
                Vinblastine_C=Vinblastine_C0*(1./2)**((np.mod(days[step-next_step]-1,7))/t_half)  #decay
                h0_local=h0_local*chemo(GVinblastine_u[i],Vinblastine_C);
                h0_dist=h0_dist*chemo(GVinblastine_u[i],Vinblastine_C);
            if days[step-next_step]==1 or days[step-next_step]==29:
                Cisplatin_C=Cisplatin_C0;
                h0_local=h0_local*chemo(GCisplatin_u[i],Cisplatin_C);
                h0_dist=h0_dist*chemo(GCisplatin_u[i],Cisplatin_C);
            elif days[step-next_step]<50 and np.mod(days[step-next_step],1)==0:
                Cisplatin_C=Cisplatin_C0*(1./2)**((np.mod(days[step-next_step]-1,28))/t_half)  #decay
                h0_local=h0_local*chemo(GCisplatin_u[i],Cisplatin_C);
                h0_dist=h0_dist*chemo(GCisplatin_u[i],Cisplatin_C);
            
            # 60 Gy  2*30 - local only
            if (days[step-next_step]<42):
                if ((np.mod(days[step-next_step]+6,7)==0 or np.mod(days[step-next_step]+5,7)==0 or np.mod(days[step-next_step]+4,7)==0 or np.mod(days[step-next_step]+3,7)==0 or np.mod(days[step-next_step]+2,7)==0)):  
                    h0_local=h0_local*radiation(current_alpha,current_alpha/10,2)
            
            #Make New Gompertz Growth+Mutation 
            intTime = [0, 1]
            cellsGrow_local = sc.integrate.odeint(GompertzMutation, h0_local, intTime, args=(Vlimit, r_sen/10, r_res/10, r_sen/10, p_mutation))        
            cellsGrow_dist  = sc.integrate.odeint(GompertzMutation, h0_dist, intTime, args=(Vlimit, r_sen/10, r_res/10, r_sen/10, p_mutation))        

            # Extract number at the end of day
            # Append result
            cellSensTreated[step,:]=np.array([float(cellsGrow_local[1,0])+float(cellsGrow_dist[1,0]), float(cellsGrow_local[1,0]), float(cellsGrow_dist[1,0])])
            cellResTreated[step,:]= np.array([float(cellsGrow_local[1,1])+float(cellsGrow_dist[1,1]), float(cellsGrow_local[1,1]), float(cellsGrow_dist[1,1])])
            cellPerTreated[step,:]= np.array([float(cellsGrow_local[1,2])+float(cellsGrow_dist[1,2]), float(cellsGrow_local[1,2]), float(cellsGrow_dist[1,2])])

            #if a comparantent has <1 cell set compartment to 0
            cellSensTreated[step,cellSensTreated[step,:]<1]=0
            cellResTreated[step,cellResTreated[step,:]<1]=0
            cellPerTreated[step,cellPerTreated[step,:]<1]=0
            
            # Update initial condition for next step
            h0_local =  np.array([cellSensTreated[step,1], cellResTreated[step,1], cellPerTreated[step,1]])
            h0_dist =   np.array([cellSensTreated[step,2], cellResTreated[step,2], cellPerTreated[step,2]])
            h0 = h0_local+h0_dist
            
            cellTotalTreated[step,:]=cellSensTreated[step,:]+cellResTreated[step,:]+cellPerTreated[step,:]
            if cellTotalTreated[step,0]<1:
                Cured =True
                t_cure = step
                break
    
    ####### TKI Until Progression #######
    
    #t definition of TIME
    tInterval = 0.1
    next_step = step
    steps_remaining = 18010-(next_step)
    tTKI = np.arange((7+wkTKIset)*7+tInterval,(7+wkTKIset)*7+steps_remaining/10+tInterval/10, tInterval)
    daysTKI = np.size(tTKI)/10  # Number of days to simulate 
    
    h0_local = np.append(h0_local,0)
    h0_dist = np.append(h0_dist,0)
    h0 = np.append(h0,0)

    if Cured==False:
        # loops in days until progression but calculates for every 0.1 day using odeint
        for step in range(next_step,next_step+np.size(tTKI),10):
            intTime = np.arange(0,1,tInterval)
            
            # get initial values for first 0.1 time point with the TKI daily bolus    
            doseErlotinibDaily = 150
            if step>next_step:
                erlotinibStart =  sc.integrate.odeint(Pharmacokinetics, h0[3], [0,1], args=([1.0182 + 0.0114*doseErlotinibDaily], ))
                erlotinibPlasma[step] = erlotinibStart[1]
                cellsTreat_local = sc.integrate.odeint(EGFRkineticsGompertz_opt2, h0_local[0:3], [0, 1], args=(Vlimit, r_sen/10, r_res/10, r_per/10, p_mutation, beta,erlotinibStart[1]))
                cellsTreat_dist = sc.integrate.odeint(EGFRkineticsGompertz_opt2, h0_dist[0:3], [0, 1], args=(Vlimit, r_sen/10, r_res/10, r_per/10, p_mutation, beta,erlotinibStart[1]))
    
                cellSensTreated[step,:]=np.array([float(cellsTreat_local[1,0])+float(cellsTreat_dist[1,0]), float(cellsTreat_local[1,0]), float(cellsTreat_dist[1,0])])
                cellResTreated[step,:]= np.array([float(cellsTreat_local[1,1])+float(cellsTreat_dist[1,1]), float(cellsTreat_local[1,1]), float(cellsTreat_dist[1,1])])
                cellPerTreated[step,:]= np.array([float(cellsTreat_local[1,2])+float(cellsTreat_dist[1,2]), float(cellsTreat_local[1,2]), float(cellsTreat_dist[1,2])])
    
            # Update initial condition for next step
            h0_local =  np.array([cellSensTreated[step,1], cellResTreated[step,1], cellPerTreated[step,1],erlotinibPlasma[step]])
            h0_dist =   np.array([cellSensTreated[step,2], cellResTreated[step,2], cellPerTreated[step,2],erlotinibPlasma[step]])
            h0 = h0_local+h0_dist; h0[3] = h0[3]/2; # because doubled when local and distant addded
            
            # Solve the ODEs (Combined Gompertz Growth/TKI Kill and & Pharmacokinetics over 1 day)
            doseinterval=0
            cellsTreat_local = sc.integrate.odeint(EGFRkineticsGompertz_opt2_Pharmacokinetics, h0_local, intTime, args=(Vlimit, r_sen, r_res, r_per, p_mutation, beta,doseinterval))
            cellsTreat_dist = sc.integrate.odeint(EGFRkineticsGompertz_opt2_Pharmacokinetics, h0_dist, intTime, args=(Vlimit, r_sen, r_res, r_per, p_mutation, beta,doseinterval))
            
            # Extract number at the end of day
            # Append result
            cellSensTreated[step+1:step+10,:]=np.transpose(np.array([(cellsTreat_local[1:10,0])+(cellsTreat_dist[1:10,0]), (cellsTreat_local[1:10,0]), (cellsTreat_dist[1:10,0])]))
            cellResTreated[step+1:step+10,:]= np.transpose(np.array([(cellsTreat_local[1:10,1])+(cellsTreat_dist[1:10,1]), (cellsTreat_local[1:10,1]), (cellsTreat_dist[1:10,1])]))
            cellPerTreated[step+1:step+10,:]= np.transpose(np.array([(cellsTreat_local[1:10,2])+(cellsTreat_dist[1:10,2]), (cellsTreat_local[1:10,2]), (cellsTreat_dist[1:10,2])]))
            erlotinibPlasma[step+1:step+10]=cellsTreat_local[1:10,3]
    
            #if a comparantent has <1 cell set compartment to 0
            cellSensTreated[step:step+10,cellSensTreated[step+9,:]<1]=0
            cellResTreated[step:step+10,cellResTreated[step+9,:]<1]=0
            cellPerTreated[step:step+10,cellPerTreated[step+9,:]<1]=0
            
            # Update initial condition for next step
            h0_local =  np.array([cellSensTreated[step+9,1], cellResTreated[step+9,1], cellPerTreated[step+9,1],erlotinibPlasma[step+9]])
            h0_dist =   np.array([cellSensTreated[step+9,2], cellResTreated[step+9,2], cellPerTreated[step+9,2],erlotinibPlasma[step+9]])
            h0 = h0_local+h0_dist; h0[3] = h0[3]/2; # because dose at index doubled when local and distant addded           
            
            cellTotalTreated=cellSensTreated+cellResTreated+cellPerTreated
            if cellTotalTreated[step+9,0]<1:
                Cured =True
                t_cure = step
                break
            
            if ((cellTotalTreated[step+9,1]>N0_total)|(cellTotalTreated[step+9,2]>G_DRRlvl)) & (step>10) & (np.mod(step,900)==0):
                break
            
    ####### regrowth after treatment if not cured or progression #######    
    steps_remaining = 18010-(step)
    if (Cured==False)&(steps_remaining>0): 
        intTime = np.arange(0,steps_remaining,1)
        cellsGrow_local = sc.integrate.odeint(GompertzMutation, h0_local[0:3], intTime, args=(Vlimit, r_sen/10, r_res/10, r_sen/10, p_mutation))        
        cellsGrow_dist  = sc.integrate.odeint(GompertzMutation, h0_dist[0:3], intTime, args=(Vlimit, r_sen/10, r_res/10, r_sen/10, p_mutation))        
        cellSensTreated[step+1:18010,:]=np.transpose(np.array([(cellsGrow_local[1:,0])+(cellsGrow_dist[1:,0]), (cellsGrow_local[1:,0]), (cellsGrow_dist[1:,0])]))
        cellResTreated[step+1:18010,:]= np.transpose(np.array([(cellsGrow_local[1:,1])+(cellsGrow_dist[1:,1]), (cellsGrow_local[1:,1]), (cellsGrow_dist[1:,1])]))
        cellPerTreated[step+1:18010,:]= np.transpose(np.array([(cellsGrow_local[1:,2])+(cellsGrow_dist[1:,2]), (cellsGrow_local[1:,2]), (cellsGrow_dist[1:,2])]))
        cellTotalTreated[step+1:18010,:] = cellSensTreated[step+1:18010,:]+cellResTreated[step+1:18010,:]+cellPerTreated[step+1:18010,:]

    ####### Comorbid Sample Russian Roullete #######    
    comorbidRate = 1-(.288/(.113+.288))**(1/60) #0.0055 mo^{-1}
    r = np.random.uniform(0,1,1)
    T_Mo_comorbid = (np.log(-(r-1))/np.log(1-comorbidRate))
        
    ####### find when patient dies and tumor progression ####### 
    # death or cure cancer
    if cellTotalTreated[18000,0]> EndT:
        ind_Tdeath=next(i for i,v in enumerate(cellTotalTreated[:,0]) if v > EndT)
        T_Mo_death=ind_Tdeath/10/30 
        T_Mo_cure = 60    
    elif Cured:
        T_Mo_cure = t_cure/10/30
        T_Mo_death = 60
    else:
        T_Mo_death = 60
        T_Mo_cure = 60
    
    # compare death or cure to randomly sampled comorbid
    if (T_Mo_comorbid<60) & (T_Mo_comorbid<T_Mo_death):
        T_Mo_death = T_Mo_comorbid
        T_Mo_cure = 60

    # find local and distant failures every 3 mo.
    if cellTotalTreated[18000,1]> N0_total:
        ind_LRR_3mo=next(i for i,v in enumerate(cellTotalTreated[900:18010:900,1]) if v > N0_total)  
        T_Mo_LRR_3mo = (ind_LRR_3mo*900+900)/10/30    
    else:
        T_Mo_LRR_3mo = 60        
    if cellTotalTreated[18000,2]> G_DRRlvl:
        ind_DRR_3mo=next(i for i,v in enumerate(cellTotalTreated[900:18010:900,2]) if v > G_DRRlvl)  
        T_Mo_DRR_3mo = (ind_DRR_3mo*900+900)/10/30    
    else:
        T_Mo_DRR_3mo = 60        
    
    T_prog = np.min([T_Mo_death,T_Mo_LRR_3mo,T_Mo_DRR_3mo])          

    out_PR = np.zeros(3)
    out_PR[0] = T_Mo_LRR_3mo; out_PR[1] = T_Mo_DRR_3mo; out_PR[2] = T_prog;
    
    return out_PR

In [13]:
if __name__ == '__main__':
    
####-------------------------------------------------------------------------------------------------------####
#                                                                                                             #
#   Run Model for N simulated patients for all treatment regimens                                             #
#                                                                                                             #
####-------------------------------------------------------------------------------------------------------####   
   
    mu_t3a =             1.914964   #stage IIIa [cm] diameter mean tumor from Geng et al.
    sig_t3a =            0.9399669  #stage IIIa [cm] diameter standard dev. tumor from Geng et al.
    mu_t3b =             2.764675   #stage IIIb [cm] diameter mean tumor from Geng et al.
    sig_t3b =            0.6871617  #stage IIIb [cm] diameter standard dev. tumor from Geng et al.
    mu_t4 =              3.869198   #stage IV [cm] diameter mean tumor from Geng et al.
    sig_t4 =             0.8822491  #stage IV [cm] diameter standard dev tumor from Geng et al.
    mu_row =             7e-05      #growth parameter [day ^-1] mean from Geng et al.
    capacity =           30         #carrying capacity [cm] diameter tumor from Geng et al.
    V_Death =            13         #death inducing [cm] diameter tumor from Geng et al.(not used in this study)

    #Chemo parameters from Geng et al.
    Vinblastine_u_mu =   0.02819574 
    Vinblastine_u_sig =  0.00068308 
    cisplatin_u_mu =     0.02819574
    cisplatin_u_sig =    0.00068308
    
    # Set Global Variables Which Will Be Called in Each Treatment Function
    global Galpha,Vlimit,EndT,GVinblastine_u,Gcisplatin_u,\
    G_mu_set,GN0_total,GN0_persister_frac,GN0_resistance,Gr_sen,Gr_per,Gr_res,Gbeta,G_wkTKIset,Gmet_scl,G_DRRlvl
    
    Vlimit=5.8e8*4/3*np.pi*(capacity/2)**3; #carrying capacity spherical tumor vol. from Geng et al.
    EndT=5.8e8*4/3*np.pi*(V_Death/2)**3; #death inducing spherical tumor vol. from Geng et al. (not used in this study)

    #########################################################################
    ####### Calilbrated Model Parameters from _FirstCalibrate Routine #######
    #########################################################################
    #Growth sigma and Initial Metatstatic Fraction
    sig_row = 0.0055
    Gmet_scl = 2e-06
    #EGFR mutant radiation distribution
    alpha_mu_egfr =  0.16
    alpha_sig_egfr = 0.32
    #WT radiation distribution
    alpha_mu_wt =  0.10
    alpha_sig_wt = 0.17
    #########################################################################

    #pre defined model params
    n_sample=16                             #number of simulated patients
    initResFrac = [0.0001,0.001,0.01,0.1]   #initial resistant fractions
    initPerFrac = [0.05, 0.2, 0.5]          #initial persistant fractions
    wk_TKI_Induction = [0,1,2,4,8,12,16]    #various inductions lengths
    G_mu_set = 1e-07                        #per -> res transition param
    G_DRRlvl = 5.8e8                        #1cc distant compartment detection limit 

    ###### create truncated log-normal and truncated normal parameter distributions ######
    # create initial stage III size distribution
    Stest_InitialDiameters=np.zeros(n_sample)
    S3a_InitialDiameters=Gettrunclognorm(mu=mu_t3a,sig=sig_t3a,lower = 0.3, upper=V_Death,num=n_sample); 
    S3b_InitialDiameters=Gettrunclognorm(mu=mu_t3b,sig=sig_t3b,lower = 0.3, upper=V_Death,num=n_sample);  
    # from PROCLAIM trial -> 47% of patients had stage IIIa and 53% had stage IIIb 
    # we hueristically create a similar stage III size distribution
    temps=np.random.uniform(0,1,size=n_sample);
    for ns in range(n_sample): 
        if temps[ns]<0.47:
            Stest_InitialDiameters[ns]=S3a_InitialDiameters[ns]; 
        else:
            Stest_InitialDiameters[ns]=S3b_InitialDiameters[ns];
    # create initial stage IV size distribution        
    Stest_InitialDiameters_IV=np.zeros(n_sample);
    Stest_InitialDiameters_IV=Gettrunclognorm(mu=mu_t4,sig=sig_t4,lower = 0.3, upper=V_Death,num=n_sample);

    # create chemo and radiation parameter distributions
    GVinblastine_u=Gettruncnorm(mu=Vinblastine_u_mu,sig=Vinblastine_u_sig,lower = 0, upper=np.inf,num=n_sample);
    GCisplatin_u=Gettruncnorm(mu=cisplatin_u_mu,sig=cisplatin_u_sig,lower = 0, upper=np.inf,num=n_sample); 
    Galpha_EGFR=Gettruncnorm(mu=alpha_mu_egfr,sig=alpha_sig_egfr,lower = 0, upper=np.inf,num=n_sample);
    Galpha_WT=Gettruncnorm(mu=alpha_mu_wt,sig=alpha_sig_wt,lower = 0, upper=np.inf,num=n_sample);

    # create growth parameter distributions
    Gr_sen = Gettruncnorm(mu=mu_row,sig=sig_row,lower = 0, upper=np.inf,num=n_sample);
    Gr_per = Gr_sen/4 # only to be activated during TKI therapy
    Gr_res = Gr_sen;
    
    # create TKI cell kill beta distribution
    Gbeta = Gettruncnorm(mu=2,sig=7,lower=1, upper=np.inf,num=n_sample) 

    #save model params 
    GMC = np.zeros([n_sample,6])
    GMC[:,0] = Stest_InitialDiameters
    GMC[:,1] = GVinblastine_u
    GMC[:,2] = GCisplatin_u
    GMC[:,3] = Galpha_EGFR
    GMC[:,4] = Gr_sen
    GMC[:,5] = Gbeta
    np.savetxt('ModelOutputs/MC_Params_Strata.csv',GMC, delimiter=',');    
    GMC_IV = np.zeros([n_sample,1])
    GMC_IV[:,0] = Stest_InitialDiameters_IV
    np.savetxt('ModelOutputs/MC_Diam_Strata_IV.csv',GMC_IV, delimiter=',');    
    GMC[:,3] = Galpha_WT
    np.savetxt('ModelOutputs/MC_Params_Strata_WT.csv',GMC, delimiter=',');   
    
    ##############################################################################
    #####                   Run Model Simulations                            #####
    ##### (note this can take very long, time with small number of patients) #####
    ##############################################################################
    model = True
    if model:
        for ipp in range(len(initPerFrac)):
            for ipr in range(len(initResFrac)):
                
                #set initial tumor distribution to stage IV for TKI only treatment until progression
                GN0_total = 5.8e8*4/3*np.pi*(Stest_InitialDiameters_IV/2)**3
                GN0_persister_frac = initPerFrac[ipp]*np.ones(n_sample)
                GN0_resistance = initResFrac[ipr]*GN0_total
                
                ##TKI only until progression 
                PatientResSave = np.zeros([n_sample,4])
                for i in range(n_sample):
                    PatientResSave[i,:]=Dynamic_TKIonlyUntilProg_MetsDRRLRR_CoMorbid_local(i)
                np.savetxt('ModelOutputs/PatientRes_TKIonlyUntilProg_ResFract{0}_PerFract{1}_mu1e-07_StageIV.csv'.format(initResFrac[ipr],initPerFrac[ipp]),PatientResSave, delimiter=',');

                #set initial tumor distribution to stage III for CRT only and TKI+CRT treatments
                GN0_total = 5.8e8*4/3*np.pi*(Stest_InitialDiameters/2)**3
                GN0_persister_frac = initPerFrac[ipp]*np.ones(n_sample)
                GN0_resistance = initResFrac[ipr]*GN0_total
                
                #set alpha to WT for CRT only WT (conncurrent and sequential, note CRT only is TKI induction length of 0 with no maintenance)
                Galpha = Galpha_WT
                G_wkTKIset = 0*np.ones(n_sample)
                
                ##CRT only concurrent - WT
                PatientResSave = np.zeros([n_sample,3])
                for i in range(n_sample):
                    PatientResSave[i,:]=Dynamic_TKIpreCRT_MetsDRRLRR_CoMorbid_local(i)
                np.savetxt('ModelOutputs/PatientRes_TKIpreCRT_ResFract{0}_PerFract{1}_mu1e-07_wkTKIset0_WT.csv'.format(initResFrac[ipr],initPerFrac[ipp]),PatientResSave, delimiter=',');

                ##CRT only sequential - WT
                PatientResSave = np.zeros([n_sample,3])
                for i in range(n_sample):
                    PatientResSave[i,:]=Dynamic_TKIpreCRT_Sequential_MetsDRRLRR_CoMorbid_local(i)
                np.savetxt('ModelOutputs/PatientRes_TKIpreCRT_Sequential_ResFract{0}_PerFract{1}_mu1e-07_wkTKIset0_WT.csv'.format(initResFrac[ipr],initPerFrac[ipp]),PatientResSave, delimiter=',');
  
                #set alpha to EGFR mut for TKI + CRT
                Galpha = Galpha_EGFR
                for ipt in range(len(wk_TKI_Induction)):
                    #set induction length
                    G_wkTKIset=wk_TKI_Induction[ipt]*np.ones(n_sample)

                    #TKI Indcution without maintenance
                    PatientResSave = np.zeros([n_sample,3])
                    for i in range(n_sample):
                        PatientResSave[i,:]=Dynamic_TKIpreCRT_MetsDRRLRR_CoMorbid_local(i)
                    np.savetxt('ModelOutputs/PatientRes_TKIpreCRT_ResFract{0}_PerFract{1}_mu1e-07_wkTKIset{2}.csv'.format(initResFrac[ipr],initPerFrac[ipp],wk_TKI_Induction[ipt]),PatientResSave, delimiter=',');
                                  
                    #TKI Indcution with maintenance until progression
                    PatientResSave = np.zeros([n_sample,3])
                    for i in range(n_sample):
                        PatientResSave[i,:]=Dynamic_TKIpreANDpostCRTuntilProg_MetsDRRLRR_CoMorbid_local(i)
                    np.savetxt('ModelOutputs/PatientRes_TKIpreAndpostCRTuntilProg_ResFract{0}_PerFract{1}_mu1e-07_wkTKIset{2}.csv'.format(initResFrac[ipr],initPerFrac[ipp],wk_TKI_Induction[ipt]),PatientResSave, delimiter=',');
                    